In [1]:
from __future__ import print_function
import os.path
import pandas as pd
import gzip
import sys
import numpy as np

sys.path.insert(0, '..')

from src.CCLE_postp_function import *
from JKBio import Datanalytics as da 
from JKBio import TerraFunction as terra
from JKBio import Helper as h
from gsheets import Sheets
from taigapy import TaigaClient
import dalmatian as dm

from sklearn.manifold import TSNE
from sklearn.neighbors import KNeighborsClassifier

from bokeh.plotting import *
from bokeh.models import HoverTool
from collections import OrderedDict
from IPython.display import Image,display



%load_ext autoreload
%autoreload 2
%load_ext rpy2.ipython
tc = TaigaClient()
output_notebook()
sheets = Sheets.from_files('~/.client_secret.json', '~/.storage.json')
replace = {'T': 'Tumor', 'N': 'Normal', 'm': 'Unknown', 'L': 'Unknown'}

you need to have installed JKBio in the same folder as ccle_processing


Loading BokehJS ...

In [2]:
samplesetname = "20Q2"
prevname="20Q1"
prevprevname ='19Q4'
prevprevversion= 18
virtual_public='public-20q2-075d'
virtual_dmc='dmc-20q2-2db6'
virtual_internal='internal-20q2-7f46'

workspace1="broad-genomics-delivery/Getz_IBM_CellLines_Exomes"
workspace2="broad-firecloud-ccle/CCLE_DepMap_WES"
workspace3="broad-genomics-delivery/CCLE_DepMap_WES"

workspace6="terra-broad-cancer-prod/CCLE_DepMap_WES"

refworkspace="broad-firecloud-ccle/DepMap_Mutation_Calling_CGA_pipeline"

rnaworkspace="broad-firecloud-ccle/DepMap_hg38_RNAseq"

source1="ibm"
source2="ccle"
source3="ccle"
source6="ccle"
source7="ibm"

refsheet_url = "https://docs.google.com/spreadsheets/d/1XkZypRuOEXzNLxVk9EOHeWRE98Z8_DBvL4PovyM01FE"
sheeturl = "https://docs.google.com/spreadsheets/d/115TUgA1t_mD32SnWAGpW9OKmJ2W5WYAOs3SuSdedpX4"

release = samplesetname

In [3]:
wm1 = dm.WorkspaceManager(workspace1)
wm2 = dm.WorkspaceManager(workspace2)
wm3 = dm.WorkspaceManager(workspace3)

wm6 = dm.WorkspaceManager(workspace6)

refwm = dm.WorkspaceManager(refworkspace)

In [4]:
ccle_refsamples = sheets.get(refsheet_url).sheets[0].to_frame()

In [ ]:
! wget https://github.com/mskcc/vcf2maf/archive/v1.6.18.zip

In [4]:
# for rnaseq
#! gsutil cp gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd50046a968/8af15313-7faf-4344-a7b7-97dd52353dc8/RNAseq/0de1da92-8821-4cbd-a9ce-96ed8b06f395/call-MergeVCFs/G41677.MV-4-11.5.g.vcf.gz temp/
path= "temp/G41677.MV-4-11.5.g.vcf.gz"
rnaseq, description = h.vcf_to_df(path)

/home/jeremie/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


{'AD': '"Allelic depths for the ref and alt alleles in the order listed"', 'DP': '"Approximate read depth; some reads may have been filtered"', 'GQ': '"Genotype Quality"', 'GT': '"Genotype"', 'PL': '"Normalized, Phred-scaled likelihoods for genotypes as defined in the VCF specification"', 'AC': '"Allele count in genotypes, for each ALT allele, in the same order as listed"', 'AF': '"Allele Frequency, for each ALT allele, in the same order as listed"', 'AN': '"Total number of alleles in called genotypes"', 'BaseQRankSum': '"Z-score from Wilcoxon rank sum test of Alt Vs. Ref base qualities"', 'DB': '"dbSNP Membership"', 'ExcessHet': '"Phred-scaled p-value for exact test of excess heterozygosity"', 'FS': '"Phred-scaled p-value using Fisher\'s exact test to detect strand bias"', 'InbreedingCoeff': '"Inbreeding coefficient as estimated from the genotype likelihoods per-sample when compared against the Hardy-Weinberg expectation"', 'MLEAC': '"Maximum likelihood expectation (MLE) for the allel

In [5]:
# for rnaseq filtered
#!gsutil cp gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd50046a968/8af15313-7faf-4344-a7b7-97dd52353dc8/RNAseq/0de1da92-8821-4cbd-a9ce-96ed8b06f395/call-VariantFiltration/G41677.MV-4-11.5.variant_filtered.vcf.gz temp/
rnaseqfilt, description = h.vcf_to_df("temp/G41677.MV-4-11.5.variant_filtered.vcf.gz")

{'AD': '"Allelic depths for the ref and alt alleles in the order listed"', 'DP': '"Approximate read depth; some reads may have been filtered"', 'GQ': '"Genotype Quality"', 'GT': '"Genotype"', 'PL': '"Normalized, Phred-scaled likelihoods for genotypes as defined in the VCF specification"', 'AC': '"Allele count in genotypes, for each ALT allele, in the same order as listed"', 'AF': '"Allele Frequency, for each ALT allele, in the same order as listed"', 'AN': '"Total number of alleles in called genotypes"', 'BaseQRankSum': '"Z-score from Wilcoxon rank sum test of Alt Vs. Ref base qualities"', 'DB': '"dbSNP Membership"', 'ExcessHet': '"Phred-scaled p-value for exact test of excess heterozygosity"', 'FS': '"Phred-scaled p-value using Fisher\'s exact test to detect strand bias"', 'InbreedingCoeff': '"Inbreeding coefficient as estimated from the genotype likelihoods per-sample when compared against the Hardy-Weinberg expectation"', 'MLEAC': '"Maximum likelihood expectation (MLE) for the allel

In [6]:
# for unfiltered wes
#!gsutil cp gs://fc-secure-012d088c-f039-4d36-bde5-ee9b1b76b912/51fcb10c-9a67-4ff9-b13c-d9667e2d49d0/HaplotypeCallerGvcf_GATK4/6611b28a-12c2-484e-a4a3-901708785f95/call-MergeGVCFs/SANGER_MV411_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE.g.vcf.gz temp/
wes, description = h.vcf_to_df("temp/SANGER_MV411_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE.g.vcf.gz")

{'AD': '"Allelic depths for the ref and alt alleles in the order listed"', 'DP': '"Approximate read depth; some reads may have been filtered"', 'GQ': '"Genotype Quality"', 'GT': '"Genotype"', 'MIN_DP': '"Minimum DP observed within the GVCF block"', 'PGT': '"Physical phasing haplotype information, describing how the alternate alleles are phased in relation to one another"', 'PID': '"Physical phasing ID information, where each unique ID within a given sample (but not across samples) connects records within a phasing group"', 'PL': '"Normalized, Phred-scaled likelihoods for genotypes as defined in the VCF specification"', 'SB': '"Per-sample component statistics which comprise the Fisher\'s Exact Test to detect strand bias."', 'BaseQRankSum': '"Z-score from Wilcoxon rank sum test of Alt Vs. Ref base qualities"', 'ClippingRankSum': '"Z-score From Wilcoxon rank sum test of Alt vs. Ref number of hard clipped bases"', 'DS': '"Were any of the samples downsampled?"', 'END': '"Stop position of th

In [7]:
# for filtered wes
#! gsutil cp gs://fc-secure-012d088c-f039-4d36-bde5-ee9b1b76b912/d953cc3f-7941-4426-bb44-a41992d57606/Cram2FilteredVcf/dc943e9d-aa73-4314-b43d-208a80319997/call-FilterVariantTranches/CDS-FInR9b_cnn_filtered.vcf.gz temp/
wesfilt, description = h.vcf_to_df("temp/CDS-FInR9b_cnn_filtered.vcf.gz")

{'AD': '"Allelic depths for the ref and alt alleles in the order listed"', 'DP': '"Approximate read depth; some reads may have been filtered"', 'GQ': '"Genotype Quality"', 'GT': '"Genotype"', 'PL': '"Normalized, Phred-scaled likelihoods for genotypes as defined in the VCF specification"', 'AC': '"Allele count in genotypes, for each ALT allele, in the same order as listed"', 'AF': '"Allele Frequency, for each ALT allele, in the same order as listed"', 'AN': '"Total number of alleles in called genotypes"', 'BaseQRankSum': '"Z-score from Wilcoxon rank sum test of Alt Vs. Ref base qualities"', 'CNN_1D': '"Log odds of being a true variant versus being false under the trained 1D Convolutional Neural Network"', 'DS': '"Were any of the samples downsampled?"', 'ExcessHet': '"Phred-scaled p-value for exact test of excess heterozygosity"', 'FS': '"Phred-scaled p-value using Fisher\'s exact test to detect strand bias"', 'InbreedingCoeff': '"Inbreeding coefficient as estimated from the genotype lik

In [8]:
# for somatic filtered wes
somwesfilt = pd.read_csv('temp/mutations.'+release+'.all.tsv', sep='\t')
somwesfilt = somwesfilt[somwesfilt.DepMap_ID=="ACH-000045"]
somwesfilt = somwesfilt.rename(columns=h.rename_mut)

/home/jeremie/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (19,29,30) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
# for somatic unfiltered wes
#!gsutil cp gs://fc-secure-012d088c-f039-4d36-bde5-ee9b1b76b912/9e3cc501-3f08-47fb-87a5-0359febb833c/CGA_Production_Analysis_Workflow/72efd16d-fa2d-4d78-b7c7-81ec2a6d08e9/call-Oncotate_Task/CDS-FInR9b.MuTect1_SNV.MuTect2_INDEL.Strelka_INDEL.annotated.maf temp/
somwes = pd.read_csv("temp/CDS-FInR9b.MuTect1_SNV.MuTect2_INDEL.Strelka_INDEL.annotated.maf", sep='\t')
somwes = somwes.rename(columns=h.rename_mut)

In [14]:
# for strelka wes
#!gsutil cp gs://fc-secure-012d088c-f039-4d36-bde5-ee9b1b76b912/9e3cc501-3f08-47fb-87a5-0359febb833c/CGA_Production_Analysis_Workflow/72efd16d-fa2d-4d78-b7c7-81ec2a6d08e9/call-Strelka/CDS-FInR9b.all.somatic.snvs.vcf temp/
strelka, description = h.vcf_to_df("temp/CDS-FInR9b.all.somatic.snvs.vcf", hasfilter=True,samples=['sample1_A','sample1_B'])
#!gsutil cp gs://fc-secure-012d088c-f039-4d36-bde5-ee9b1b76b912/9e3cc501-3f08-47fb-87a5-0359febb833c/CGA_Production_Analysis_Workflow/72efd16d-fa2d-4d78-b7c7-81ec2a6d08e9/call-Strelka/CDS-FInR9b.all.somatic.indels.vcf temp/
strelka2, description = h.vcf_to_df("temp/CDS-FInR9b.all.somatic.indels.vcf",samples=['sample1_A','sample1_B'],hasfilter=True)
strelka = pd.concat([strelka,strelka2], ignore_index=True,sort=False).sort_values(by=['chr','pos'])

{'QSS': '"Quality score for any somatic snv, ie. for the ALT allele to be present at a significantly different frequency in the tumor and normal"', 'TQSS': '"Data tier used to compute QSS"', 'NT': '"Genotype of the normal in all data tiers, as used to classify somatic variants. One of {ref,het,hom,conflict}."', 'QSS_NT': '"Quality score reflecting the joint probability of a somatic variant and NT"', 'TQSS_NT': '"Data tier used to compute QSS_NT"', 'SGT': '"Most likely somatic genotype excluding normal noise states"', 'SOMATIC': '"Somatic mutation"', 'DP': '"Read depth for tier1 (used+filtered)"', 'FDP': '"Number of basecalls filtered from original read depth for tier1"', 'SDP': '"Number of reads with deletions spanning this site at tier1"', 'SUBDP': '"Number of reads below tier1 mapping quality threshold aligned across this site"', 'AU': '"Number of \'A\' alleles used in tiers 1,2"', 'CU': '"Number of \'C\' alleles used in tiers 1,2"', 'GU': '"Number of \'G\' alleles used in tiers 1,2"

In [15]:
# for mutect1 wes
#!gsutil cp gs://fc-secure-012d088c-f039-4d36-bde5-ee9b1b76b912/9e3cc501-3f08-47fb-87a5-0359febb833c/CGA_Production_Analysis_Workflow/72efd16d-fa2d-4d78-b7c7-81ec2a6d08e9/call-Gather_Task/CDS-FInR9b.MuTect1.call_stats.txt temp/
mutect1 = pd.read_csv('temp/CDS-FInR9b.MuTect1.call_stats.txt', sep='\t', header=1, index_col=None)
mutect1 = mutect1.rename(h.rename_mut)

/home/jeremie/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [16]:
# for mutect 2 wes
#!gsutil cp gs://fc-secure-012d088c-f039-4d36-bde5-ee9b1b76b912/9e3cc501-3f08-47fb-87a5-0359febb833c/CGA_Production_Analysis_Workflow/72efd16d-fa2d-4d78-b7c7-81ec2a6d08e9/call-Gather_Task/CDS-FInR9b.MuTect2.call_stats.vcf temp/
mutect2, description = h.vcf_to_df('temp/CDS-FInR9b.MuTect2.call_stats.vcf', hasfilter=True)

{'AD': '"Allelic depths for the ref and alt alleles in the order listed"', 'AF': '"Allele fractions of alternate alleles in the tumor"', 'DP': '"Approximate read depth; some reads may have been filtered"', 'F1R2': '"Count of reads in F1R2 pair orientation supporting each allele"', 'F2R1': '"Count of reads in F2R1 pair orientation supporting each allele"', 'GQ': '"Genotype Quality"', 'GT': '"Genotype"', 'MBQ': '"median base quality"', 'MFRL': '"median fragment length"', 'MMQ': '"median mapping quality"', 'MPOS': '"median distance from end of read"', 'PGT': '"Physical phasing haplotype information, describing how the alternate alleles are phased in relation to one another"', 'PID': '"Physical phasing ID information, where each unique ID within a given sample (but not across samples) connects records within a phasing group"', 'PL': '"Normalized, Phred-scaled likelihoods for genotypes as defined in the VCF specification"', 'SA_MAP_AF': '"MAP estimates of allele fraction given z"', 'SA_POST

In [17]:
# for unfiltered wgs
# for filtered wgs
#! gsutil cp gs://fc-secure-bd7b8bc9-f665-4269-997e-5a402088a369/77dbe69a-509c-4618-a2bf-f809c1aea47a/liftover_workflows/2950a03d-c092-4355-b261-2e6eb50fca41/call-liftover/ccle_ACH-000045_liftover.vcf temp/
wgsfilt, description = h.vcf_to_df('temp/ccle_ACH-000045_liftover.vcf')

{'AD': '"Allelic depths for the ref and alt alleles in the order listed"', 'DP': '"Approximate read depth; some reads may have been filtered"', 'GQ': '"Genotype Quality"', 'GT': '"Genotype"', 'PL': '"Normalized, Phred-scaled likelihoods for genotypes as defined in the VCF specification"', 'AC': '"Allele count in genotypes, for each ALT allele, in the same order as listed"', 'AF': '"Allele Frequency, for each ALT allele, in the same order as listed"', 'AN': '"Total number of alleles in called genotypes"', 'BaseQRankSum': '"Z-score from Wilcoxon rank sum test of Alt Vs. Ref base qualities"', 'CNN_1D': '"Log odds of being a true variant versus being false under the trained 1D Convolutional Neural Network"', 'DS': '"Were any of the samples downsampled?"', 'END': '"Stop position of the interval"', 'ExcessHet': '"Phred-scaled p-value for exact test of excess heterozygosity"', 'FS': '"Phred-scaled p-value using Fisher\'s exact test to detect strand bias"', 'InbreedingCoeff': '"Inbreeding coef

In [18]:
data= {'wgsfilt': wgsfilt, 'mutect2': mutect2, 'mutect1': mutect1, 'strelka': strelka, 'somwes': somwes, 'somwesfilt': somwesfilt, 'wes': wes, 'rnaseq': rnaseq, 'rnaseqfilt': rnaseqfilt, 'wesfilt': wesfilt}

In [19]:
mutset = {}
for i,v in data.items():
    chrom = v.chr.tolist()
    alt = v.alt.tolist()
    pos = v.pos.tolist()
    mutset[i] = set([str(chrom[i])+'_'+str(pos[i]) for i, a in enumerate(alt)])

AttributeError: 'DataFrame' object has no attribute 'chr'

In [306]:
len(mutset['wgsfilt'] & mutset['wesfilt']) / len(mutset['wesfilt']), len(mutset['wgsfilt']), len(mutset['wesfilt'])

(0.8340264369523244, 4384476, 26932)

In [209]:
len(mutset['rnaseqfilt']  & mutset['wesfilt']) / len(mutset['wesfilt'])

0.4082132778850438

In [210]:
len(mutset['rnaseq']  & mutset['wesfilt']) / len(mutset['wesfilt'])

0.4082132778850438

In [211]:
len(mutset['rnaseqfilt']  & mutset['rnaseq']) / len(mutset['rnaseq'])

1.0

In [214]:
len(mutset['wes']  & mutset['wesfilt']) / len(mutset['wesfilt'])

0.9542551611465914

In [215]:
len(mutset['wes']  & mutset['wesfilt']) / len(mutset['wes'])
# how many mutations is the cnn filtering..

0.006469112880733723

In [217]:
len(mutset['wesfilt']), len(mutset['wes'])

(26932, 3972724)

In [220]:
len(mutset['strelka']), len(mutset['mutect1']), len(mutset['mutect2'])
# how many mutations each contain and their similarity between each caller in cga

(51150, 32547, 2276)

In [221]:
len(mutset['strelka'] & mutset['mutect1'] & mutset['mutect2'])

655

In [234]:
len(mutset['somwes']), len(mutset['somwesfilt']), len(mutset['somwesfilt'] & mutset['somwes'])
# how many mutations each contain and their similarity between cga middle and cga end

(789, 222, 154)

In [236]:
len(mutset['somwes'] & mutset['mutect1']) / len(mutset['mutect1']), len(mutset['somwesfilt'] & mutset['mutect1']) / len(mutset['mutect1'])

(0.008695117829600271, 0.005131041263403693)

In [237]:
len(mutset['somwes'] & mutset['mutect2']) / len(mutset['mutect2']), len(mutset['somwesfilt'] & mutset['mutect2']) / len(mutset['mutect2'])

(0.11862917398945519, 0.06239015817223199)

In [238]:
len(mutset['somwes'] & mutset['strelka']) / len(mutset['strelka']), len(mutset['somwesfilt'] & mutset['strelka']) / len(mutset['strelka'])

(0.009657869012707723, 0.0028543499511241447)

In [244]:
len(mutset['somwes'] & mutset['wes']) / len(mutset['somwes']), len(mutset['wes'])

(0.4372623574144487, 3972724)

In [245]:
len(mutset['somwes'] & mutset['wesfilt']) / len(mutset['somwes']), len(mutset['wesfilt'])

(0.3650190114068441, 26932)

In [247]:
len(mutset['somwesfilt'] & mutset['wesfilt']) / len(mutset['somwesfilt']), len(mutset['wesfilt'])
# good agreement between wes hapcall and wes cga and we can see how much we recover once bad mutations get filtered!

(0.7297297297297297, 26932)

In [ ]:
len(mutset['somwes'] & mutset['wesfilt']), len(mutset['somwesfilt'] & mutset['wesfilt'])

In [310]:
len(mutset['somwes'] & mutset['wgsfilt']) / len(mutset['somwes'])

0.5906210392902408

In [309]:
len(mutset['somwesfilt'] & mutset['wgsfilt']) / len(mutset['somwesfilt'])
# good agreement between wgs hapcall and wes cga

0.8333333333333334

In [248]:
len(mutset['somwesfilt'] & mutset['wes']) / len(mutset['somwesfilt']), len(mutset['wes'])
# good agreement between wes hapcall and wes cga

(0.7477477477477478, 3972724)

In [308]:
#we don't have a good similarity wes/wgs, can it be because of slight differences in the base distances? can it be because of the mutation caller?
len(set([i for i in mutset['wesfilt']]) & set([i for i in mutset['wgsfilt']]))/len(mutset['wesfilt'])

0.8340264369523244

In [258]:
def locify(dat):
    return ['chr'+d.split('_')[0]+':'+str(int(d.split('_')[1])-40)+'-'+str(int(d.split('_')[1])+40) for d in dat]

In [278]:
'chr6:27420132-27420212':'rna',
'chr5:32089849-32089929':'del',
'chr4:100574667-100574747':'wgs',
'chr7:100643254-100643334':'wgs',
'chr15:89070794-89070874':'wgs',
'chr19:36120039-36120119':'snp',
'chr20:31196288-31196368':'snp',
'chr15:42343066-42343146':False,
'chr11:6568654-6568734':'snp',

'chr9:24543236-24543316':'snp',
'chr15:23261902-23261982':'snp',
'chr4:84255808-84255888':True,
'chr13:76055619-76055699':'snp',
'chrX:18969242-18969322':,
'chr4:36152613-36152693':,
'chr19:50354695-50354775':,
'chr4:100574668-100574748':,
'chr6:110763839-110763919':,
'chr7:100648111-100648191':,
'chr5:70858221-70858301':,
'chr3:120489593-120489673':,
'chr2:27601639-27601719':,
'chr4:77184865-77184945':,

SyntaxError: invalid syntax (<ipython-input-278-867e7cbb5ced>, line 1)

In [259]:
locify(mutset['somwesfilt'] - mutset['wesfilt'])

['chr6:27420132-27420212',
 'chr5:32089849-32089929',
 'chr4:100574667-100574747',
 'chr7:100643254-100643334',
 'chr15:89070794-89070874',
 'chr19:36120039-36120119',
 'chr20:31196288-31196368',
 'chr15:42343066-42343146',
 'chr11:6568654-6568734',
 'chr9:24543236-24543316',
 'chr15:23261902-23261982',
 'chr4:84255808-84255888',
 'chr13:76055619-76055699',
 'chrX:18969242-18969322',
 'chr4:36152613-36152693',
 'chr19:50354695-50354775',
 'chr4:100574668-100574748',
 'chr6:110763839-110763919',
 'chr7:100648111-100648191',
 'chr5:70858221-70858301',
 'chr3:120489593-120489673',
 'chr2:27601639-27601719',
 'chr4:77184865-77184945',
 'chr1:248084985-248085065',
 'chr11:120107369-120107449',
 'chr2:230724170-230724250',
 'chr8:1628673-1628753',
 'chr6:96985332-96985412',
 'chrX:47099210-47099290',
 'chr2:120187519-120187599',
 'chr2:100015229-100015309',
 'chr1:35452749-35452829',
 'chr11:376785-376865',
 'chr18:10750098-10750178',
 'chr14:64970485-64970565',
 'chr16:28962524-28962604',
 

In [276]:
def subset(df,mut):
    return pd.concat([df[(df.chr==val.split('_')[0]) & (df.pos==int(val.split('_')[1]))] for val in mut])

In [317]:
a = subset(data['somwesfilt'],mutset['somwesfilt'] - mutset['wesfilt'])
print(60-a.SangerRecalibWES_AC.isna().sum())
a[~a.SangerRecalibWES_AC.isna()]

20


,Hugo_Symbol,Entrez_Gene_Id,NCBI_Build,chr,pos,End_position,Strand,Variant_Classification,Variant_Type,ref,...,ExAC_AF,CGA_WES_AC,SangerWES_AC,SangerRecalibWES_AC,RNAseq_AC,HC_AC,RD_AC,WGS_AC,Variant_annotation,DepMap_ID
826976,PDZD2,23037,37,5,32089889,32089897,+,In_Frame_Del,DEL,GCGACAGAC,...,NaN,NaN,7:0,9:24,NaN,NaN,NaN,15:20,other non-conserving,ACH-000045
930038,RBM42,79171,37,19,36120079,36120079,+,Silent,SNP,G,...,NaN,NaN,NaN,4:8,NaN,NaN,NaN,NaN,silent,ACH-000045
956348,RP11-410N8.4,0,37,20,31196328,31196328,+,Missense_Mutation,SNP,A,...,NaN,NaN,4:6,4:7,NaN,NaN,NaN,16:23,other non-conserving,ACH-000045
309536,DNHD1,144132,37,11,6568694,6568694,+,Silent,SNP,C,...,NaN,NaN,29:29,30:29,22:24,NaN,NaN,26:17,silent,ACH-000045
444857,GOLGA8I,283796,37,15,23261942,23261942,+,Missense_Mutation,SNP,A,...,0.000034,NaN,16:73,16:90,NaN,NaN,NaN,NaN,other non-conserving,ACH-000045
1105410,TBC1D4,9882,37,13,76055659,76055659,+,Missense_Mutation,SNP,A,...,NaN,NaN,NaN,4:16,NaN,NaN,NaN,NaN,other non-conserving,ACH-000045
836283,PHKA2,5256,37,X,18969282,18969283,+,Frame_Shift_Del,DEL,GG,...,NaN,NaN,17:2,18:6,11:25,NaN,NaN,3:7,damaging,ACH-000045
690021,MUC12,10071,37,7,100648151,100648151,+,Silent,SNP,G,...,NaN,NaN,56:71,54:70,NaN,NaN,NaN,13:16,silent,ACH-000045
377082,FAM47E,100129583,37,4,77184905,77184905,+,Missense_Mutation,SNP,G,...,0.000039,NaN,50:70,49:68,NaN,NaN,NaN,13:13,other non-conserving,ACH-000045
778104,OR2T8,343172,37,1,248085025,248085025,+,Frame_Shift_Del,DEL,T,...,NaN,NaN,6:2,8:19,NaN,NaN,NaN,NaN,damaging,ACH-000045


####  we have 60-40=20 that the old pipeline has found but not us. of these 20; 7 were suspicious as they were not confirmed by any other sequencing methods, had low depth etc.

In [319]:
res = subset(data['somwesfilt'],mutset['somwesfilt'] & mutset['wesfilt'])
print(res.SangerRecalibWES_AC.isna().sum())
res[res.SangerRecalibWES_AC.isna()]

16


,Hugo_Symbol,Entrez_Gene_Id,NCBI_Build,chr,pos,End_position,Strand,Variant_Classification,Variant_Type,ref,...,ExAC_AF,CGA_WES_AC,SangerWES_AC,SangerRecalibWES_AC,RNAseq_AC,HC_AC,RD_AC,WGS_AC,Variant_annotation,DepMap_ID
527683,ISOC1,51015,37,5,128430479,128430479,+,Missense_Mutation,SNP,C,...,NaN,NaN,NaN,NaN,47:47,NaN,NaN,11:7,other non-conserving,ACH-000045
729754,NEURL,0,37,10,105344322,105344322,+,Missense_Mutation,SNP,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7:10,other non-conserving,ACH-000045
138731,C1orf122,127687,37,1,38274362,38274362,+,Silent,SNP,A,...,0.000010,NaN,NaN,NaN,102:85,NaN,NaN,14:10,silent,ACH-000045
247396,CPVL,54504,37,7,29035414,29035414,+,Missense_Mutation,SNP,T,...,NaN,NaN,21:29,NaN,NaN,NaN,NaN,18:12,other non-conserving,ACH-000045
385827,FBLN2,2199,37,3,13655515,13655515,+,Missense_Mutation,SNP,G,...,NaN,NaN,15:23,NaN,NaN,NaN,NaN,10:18,other non-conserving,ACH-000045
247398,CPVL,54504,37,7,29035415,29035415,+,Silent,SNP,T,...,NaN,NaN,20:29,NaN,NaN,NaN,NaN,17:12,silent,ACH-000045
1172171,TRPM5,29850,37,11,2434364,2434364,+,Splice_Site,SNP,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20:17,damaging,ACH-000045
291218,DHX36,170506,37,3,154018843,154018843,+,Nonsense_Mutation,SNP,C,...,NaN,NaN,46:45,NaN,NaN,NaN,NaN,8:24,damaging,ACH-000045
291221,DHX36,170506,37,3,154018844,154018844,+,Missense_Mutation,SNP,T,...,NaN,NaN,46:43,NaN,NaN,NaN,NaN,9:24,other non-conserving,ACH-000045
335550,EHMT1,79813,37,9,140707977,140707977,+,Missense_Mutation,SNP,C,...,NaN,NaN,12:12,NaN,147:142,NaN,NaN,18:17,other non-conserving,ACH-000045


In [ ]:
# we have found 16 mutations that the old pipeline did not fouund, of these, only one was only confirmed by one sequencing method

In [287]:
subset(data['somwesfilt'],mutset['rnaseqfilt'] & mutset['somwesfilt']).RNAseq_AC.isna().sum() #113

12

In [293]:
subset(data['somwesfilt'],mutset['somwesfilt'] - mutset['rnaseqfilt']).RNAseq_AC.isna().sum() #113

104

In [ ]:
# we have found 12 more rnaseq mutations and lost 9 rnaseq mutations

In [324]:
res = subset(data['somwesfilt'],mutset['somwesfilt'] - mutset['wgsfilt'])
print(37-res.WGS_AC.isna().sum())

27


In [323]:
res

,Hugo_Symbol,Entrez_Gene_Id,NCBI_Build,chr,pos,End_position,Strand,Variant_Classification,Variant_Type,ref,...,ExAC_AF,CGA_WES_AC,SangerWES_AC,SangerRecalibWES_AC,RNAseq_AC,HC_AC,RD_AC,WGS_AC,Variant_annotation,DepMap_ID
1267443,ZNF184,7738,37,6,27420172,27420173,+,Frame_Shift_Ins,INS,-,...,0.000008,NaN,NaN,NaN,6:28,NaN,NaN,NaN,damaging,ACH-000045
826976,PDZD2,23037,37,5,32089889,32089897,+,In_Frame_Del,DEL,GCGACAGAC,...,NaN,NaN,7:0,9:24,NaN,NaN,NaN,15:20,other non-conserving,ACH-000045
3313,ABCA13,154664,37,7,48559709,48559709,+,Silent,SNP,C,...,0.000008,NaN,34:148,33:148,NaN,NaN,NaN,NaN,silent,ACH-000045
930038,RBM42,79171,37,19,36120079,36120079,+,Silent,SNP,G,...,NaN,NaN,NaN,4:8,NaN,NaN,NaN,NaN,silent,ACH-000045
871718,POU6F2,11281,37,7,39379285,39379285,+,Missense_Mutation,SNP,C,...,0.000042,NaN,NaN,47:55,NaN,NaN,NaN,NaN,other non-conserving,ACH-000045
444857,GOLGA8I,283796,37,15,23261942,23261942,+,Missense_Mutation,SNP,A,...,0.000034,NaN,16:73,16:90,NaN,NaN,NaN,NaN,other non-conserving,ACH-000045
1105410,TBC1D4,9882,37,13,76055659,76055659,+,Missense_Mutation,SNP,A,...,NaN,NaN,NaN,4:16,NaN,NaN,NaN,NaN,other non-conserving,ACH-000045
836283,PHKA2,5256,37,X,18969282,18969283,+,Frame_Shift_Del,DEL,GG,...,NaN,NaN,17:2,18:6,11:25,NaN,NaN,3:7,damaging,ACH-000045
71924,ARAP2,116984,37,4,36152653,36152653,+,Frame_Shift_Del,DEL,T,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7:25,damaging,ACH-000045
1018133,SLC22A16,85413,37,6,110763879,110763880,+,Frame_Shift_Ins,INS,-,...,0.000016,NaN,NaN,NaN,11:60,NaN,NaN,NaN,damaging,ACH-000045


### we have high allele freq mut which seems it might be that they get filtered by the cnn. two seemed quite shady